How Temperature Affects Coulomb Counting
a. Battery Capacity Is Temperature-Dependent
Battery capacity 
𝐶
C decreases at low temperatures because:

Electrochemical reactions slow down

Internal resistance increases

Some capacity becomes inaccessible

Example:
A 100Ah battery might only deliver ~80Ah at -10°C.

➡️ Coulomb counting will overestimate SoC at low temps, since it assumes a fixed 100Ah.

b. Current Sensor Drift
Shunt-based or Hall-effect current sensors can drift with temperature, leading to:

Small, systematic errors in current

Which accumulate into significant SoC errors over time

➡️ Thermal compensation is often needed in sensor electronics.

c. Self-Discharge and Parasitic Losses
These increase with temperature:

Coulomb counting doesn't track self-discharge

High temperatures make this worse

➡️ Coulomb counting underestimates SoC at high temps if not corrected.

d. Internal Resistance & Peukert Effect
Though less significant for simple Coulomb counting:

Resistance increases at low T

High internal resistance causes greater power loss

In [1]:
def temperature_corrected_capacity(nominal_capacity, temperature_c):
    """
    Adjust capacity based on temperature.
    Capacity decreases ~0.5% per °C below 25°C.
    """
    if temperature_c < 25:
        reduction_factor = 1 - 0.005 * (25 - temperature_c)
    else:
        reduction_factor = 1  # No gain above 25°C (can be refined)
    return nominal_capacity * reduction_factor


def ah_integration_temp(soc_initial, current, time_step, nominal_capacity, temperature_c):
    """
    Estimate SoC using Ah integration with temperature-corrected capacity.
    soc_initial: Initial SoC (0 to 1)
    current: Current in A (positive for charge, negative for discharge)
    time_step: Time interval in hours
    nominal_capacity: Battery capacity in Ah at 25°C
    temperature_c: Current temperature in °C
    """
    capacity_effective = temperature_corrected_capacity(nominal_capacity, temperature_c)
    charge_change = current * time_step  # Ah
    soc_change = charge_change / capacity_effective
    soc_new = soc_initial + soc_change
    return max(0, min(1, soc_new))  # Clamp SoC between 0 and 1


# Example usage
soc_0 = 0.8          # 80%
current = -5         # 5A discharge
time_step = 2        # 2 hours
capacity = 100       # 100 Ah at 25°C
temperature = 10     # 10°C

soc = ah_integration_temp(soc_0, current, time_step, capacity, temperature)
print(f"New SoC at {temperature}°C: {soc * 100:.1f}%")


New SoC at 10°C: 69.2%
